# Setting up an external computer

## Aim

We can use aiida to submit jobs to an external computer. In this notebook we will: configure a profile, setup a computer, configure the SSH transport layer and create a code.

## Setup

For this tutorial we will assume you have:

<ul>
        <li>Aiida setup and can run jobs locally</li>
        <li>Quantum Espresso setup on the remote computer</li>
</ul>

### Profile

For the profile setup, you can either use SQLite or Postgres. For the sake of simplicity we will use `verdi presto` to setup the profile as it sets up all the base steps we need like: creating a profile, setting up localhost computer and a database with SQLite. The [Aiida](https://aiida.readthedocs.io/projects/aiida-core/en/stable/installation/guide_complete.html#create-a-profile) docs go into more detail in regards to setting up a profile.

In [ ]:
! verdi presto -p scarf

### Computer

Now we setup the [computer](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/run_codes.html#computer-setup) with the following settings. ( **NOTE**: This setup is to connect to SCARF, you should change the label, description, hostname and workdir respectively inline with your remote system settings ) <br>
(**Optional**) We are loading the module we want in prepend_text. As we will be needing Quantum Espresso to load the code in the later steps we will load it now, otherwise you can leave it empty `prepend_text: ''`.

In [ ]:
%%writefile ~/computer.yaml
label: [scarf]
description: [https://www.scarf.rl.ac.uk/index.html]
hostname: [ui1.scarf.rl.ac.uk]
transport: core.ssh
scheduler: core.slurm
shebang: '#!/bin/bash'
work_dir: [/work4/scd/scarf/aiida]
mpirun_command: srun -u -n {tot_num_mpiprocs}
mpiprocs_per_machine: 32
use_double_quotes: False
prepend_text: |
  module load amd-modules
  module load QuantumESPRESSO/7.2-foss-2023a
append_text: ''

Run the following commands making sure to change the setting in `verdi computer configure ...` in square brackets with your settings. The [Aiida](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/run_codes.html#how-to-set-up-a-computer) docs go over how to configure a computer in more detail.

In [ ]:
! verdi computer setup -n --config ~/computer.yaml

In [ ]:
! verdi computer configure core.ssh [scarf] -n --username [scarf] --key-filename ['/path/to/ssh/key'] --key-policy AutoAddPolicy --safe-interval 30.0

Once the compuuter is setup, you can test the connection

In [ ]:
! verdi computer test [scarf]

### External Code

If you want to run a calculation that is only available on the external machine, you can set up a code which accesses the external library. `verdi code setup -h` goes into more detail about what each of options does. For more information read the [Aiida docs](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/run_codes.html#how-to-set-up-a-computer)

In [ ]:
%%writefile ~/qe_code.yaml
label: qe
description: Quantum ESPRESSO pw.x
default_calc_job_plugin: quantumespresso.pw
computer: scarf
filepath_executable: /work4/scd/scarf562/eb-amd/software/QuantumESPRESSO/7.2-foss-2023a/bin/pw.x
use_double_quotes: true
prepend_text: ''
append_text: ''

In [ ]:
! verdi code create core.code.installed -n --config ~/qe_code.yaml